## 问题描述

使用 PINN 和 OpenFOAM 数据，在给定流场数据的情况下，求解流体粘度的逆问题。

## 练习

本题的主要目标是使用 PINN 正确地公式化问题。为了实现这一点，你必须成功完成以下部分：
1. 吸收 OpenFOAM 数据
2. 建立正确的方程残差以最小化残差
3. 创建神经网络并求解逆问题

OpenFOAM 模拟中的粘度设置为 $0.02 \text{ }m^2/s$。成功完成本题后，应推断出相同的粘度，误差在 10% 以内。之前部分用于验证的 OpenFOAM 数据现在将用作训练输入。

让我们从导入所需的软件包开始。

**注意：您需要编辑位于 `chip_2d/` 目录中的 [`chip_2d_inverse_template.py`](../../source_code/chip_2d/chip_2d_inverse_template.py) 脚本。**

```python
import torch
import numpy as np
from sympy import Symbol, Eq

import modulus
from modulus.hydra import to_absolute_path, ModulusConfig, instantiate_arch
from modulus.utils.io import csv_to_dict
from modulus.solver import Solver
from modulus.domain import Domain
from modulus.geometry.primitives_2d import Rectangle, Line, Channel2D
from modulus.utils.sympy.functions import parabola
from modulus.eq.pdes.navier_stokes import NavierStokes
from modulus.eq.pdes.basic import NormalDotVec
from modulus.domain.constraint import (
    PointwiseConstraint,
)
from modulus.domain.monitor import PointwiseMonitor
from modulus.key import Key
from modulus.node import Node
```

对于这个问题，由于我们只关注吸收 OpenFOAM 数据并从中得出推论，因此无需创建几何体。此外，在设置方程时，不要忘记分离相应变量的梯度。

```python
#TODO: Replace all the placeholders with appropriate values

@modulus.main(config_path="conf", config_name="config_inverse")
def run(cfg: ModulusConfig) -> None:

    # make list of nodes to unroll graph on
    ns = NavierStokes(nu="nu", rho=1.0, dim=2, time=False)
    flow_net = instantiate_arch(
        input_keys=[Key("x"), Key("y")],
        output_keys=[Key("u"), Key("v"), Key("p")],
        cfg=cfg.arch.fully_connected,
    )
    inverse_net = instantiate_arch(
        input_keys=[Key("x"), Key("y")],
        output_keys=[Key("nu")],
        cfg=cfg.arch.fully_connected,
    )

    nodes = (
        ns.make_nodes(
            detach_names=[placeholder]
        )
        + [flow_net.make_node(name="flow_network", jit=cfg.jit)]
        + [inverse_net.make_node(name="inverse_network", jit=cfg.jit)]
    )

    # add constraints to solver
    # data
    mapping = {"Points:0": "x", "Points:1": "y", "U:0": "u", "U:1": "v", "p": "p"}
    openfoam_var = csv_to_dict(to_absolute_path("openfoam/2D_chip_fluid0.csv"), mapping)
    openfoam_var["x"] -= 2.5  # normalize pos
    openfoam_var["y"] -= 0.5
    openfoam_invar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["x", "y"]
    }
    openfoam_outvar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["u", "v", "p"]
    }
    openfoam_outvar_numpy["continuity"] = placeholder
    openfoam_outvar_numpy["momentum_x"] = placeholder
    openfoam_outvar_numpy["momentum_y"] = placeholder

    # make domain
    domain = Domain()

    # data and pdes
    data = PointwiseConstraint.from_numpy(
        nodes=nodes,
        invar=openfoam_invar_numpy,
        outvar=openfoam_outvar_numpy,
        batch_size=cfg.batch_size.data,
    )
    domain.add_constraint(data, name="Data")
```

创建模型预测的平均粘度监测器：

```python
    # add monitors
    monitor = PointwiseMonitor(
        openfoam_invar_numpy,
        output_names=["nu"],
        metrics={"mean_nu": lambda var: torch.mean(var["nu"])},
        nodes=nodes,
    )
    domain.add_monitor(monitor)
```

现在通过创建求解器来解决我们的问题来完成代码的最后一部分。

```python
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()


if __name__ == "__main__":
    run()
```